# Example 1: Multi-Model Forecast Comparison

**Phase 4 Features Showcased:**
- ✅ Dynamic Layout Controls (adjust ncol/nrow)
- ✅ Label Configuration (selective metadata display)
- ✅ Filters (multi-criteria filtering)
- ✅ Sorts (performance-based sorting)
- ✅ Views (save/load filter combinations)
- ✅ Search (find specific models/series)
- ✅ Export (download filtered results)

## Use Case

Compare forecasting performance across different models (ARIMA, Prophet, ETS, XGBoost) for multiple time series representing product sales across different categories.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from trelliscope import Display
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 1. Generate Synthetic Forecasting Data

In [2]:
def generate_time_series(n_periods=100, trend_strength=0.1, seasonal_strength=5, noise_std=2):
    """Generate synthetic time series with trend, seasonality, and noise."""
    t = np.arange(n_periods)
    trend = trend_strength * t
    seasonal = seasonal_strength * np.sin(2 * np.pi * t / 12)  # Monthly seasonality
    noise = np.random.normal(0, noise_std, n_periods)
    return trend + seasonal + noise + 50  # Base level of 50

def generate_forecast(actual, model_name):
    """Generate model-specific forecast with characteristic errors."""
    n = len(actual)
    
    # Model-specific characteristics
    model_params = {
        'ARIMA': {'lag': 1, 'noise': 1.5, 'trend_capture': 0.9},
        'Prophet': {'lag': 0, 'noise': 1.8, 'trend_capture': 0.95},
        'ETS': {'lag': 2, 'noise': 2.0, 'trend_capture': 0.85},
        'XGBoost': {'lag': 1, 'noise': 1.2, 'trend_capture': 0.92}
    }
    
    params = model_params[model_name]
    
    # Create forecast with lag and noise
    forecast = np.roll(actual, params['lag']) * params['trend_capture']
    forecast += np.random.normal(0, params['noise'], n)
    
    return forecast

# Generate data for 20 series × 4 models = 80 panels
products = [f'Product_{chr(65+i)}' for i in range(20)]  # Product_A to Product_T
categories = ['Electronics', 'Apparel', 'Food', 'Home'] * 5  # Distribute across categories
models = ['ARIMA', 'Prophet', 'ETS', 'XGBoost']

print(f"Generating forecast data for {len(products)} products × {len(models)} models = {len(products) * len(models)} panels...")

Generating forecast data for 20 products × 4 models = 80 panels...


## 2. Create Forecast Visualizations

In [3]:
def create_forecast_plot(actual, forecast, product, model, rmse, mae):
    """Create publication-quality forecast visualization."""
    fig, ax = plt.subplots(figsize=(10, 5))
    
    t = np.arange(len(actual))
    
    # Plot actual and forecast
    ax.plot(t, actual, label='Actual', color='#2C3E50', linewidth=2.5, alpha=0.8)
    ax.plot(t, forecast, label='Forecast', color='#E74C3C', linewidth=2, linestyle='--', alpha=0.9)
    
    # Add confidence interval (simplified)
    std = np.std(actual - forecast)
    ax.fill_between(t, forecast - 1.96*std, forecast + 1.96*std, 
                     color='#E74C3C', alpha=0.15, label='95% CI')
    
    # Styling
    ax.set_title(f'{product} - {model}\nRMSE: {rmse:.2f} | MAE: {mae:.2f}', 
                 fontsize=13, fontweight='bold', pad=15)
    ax.set_xlabel('Time Period', fontsize=11)
    ax.set_ylabel('Sales', fontsize=11)
    ax.legend(loc='upper left', framealpha=0.95, fontsize=10)
    ax.grid(True, alpha=0.25, linestyle=':')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig

# Generate all forecast panels and collect data
data_rows = []
panel_count = 0

for i, product in enumerate(products):
    # Generate actual time series for this product
    actual = generate_time_series(
        n_periods=100,
        trend_strength=np.random.uniform(0.05, 0.15),
        seasonal_strength=np.random.uniform(3, 7),
        noise_std=np.random.uniform(1.5, 3)
    )
    
    category = categories[i]
    
    for model in models:
        # Generate forecast
        forecast = generate_forecast(actual, model)
        
        # Calculate metrics
        errors = actual - forecast
        rmse = np.sqrt(np.mean(errors**2))
        mae = np.mean(np.abs(errors))
        mape = np.mean(np.abs(errors / actual)) * 100
        
        # Simulate training time (faster models = less complex)
        training_times = {'ARIMA': 0.5, 'Prophet': 2.0, 'ETS': 0.3, 'XGBoost': 1.5}
        training_time = training_times[model] * (1 + np.random.uniform(-0.2, 0.2))
        
        # Create visualization
        fig = create_forecast_plot(actual, forecast, product, model, rmse, mae)
        
        # Collect data
        data_rows.append({
            'panel': fig,
            'product': product,
            'category': category,
            'model': model,
            'rmse': rmse,
            'mae': mae,
            'mape': mape,
            'training_time': training_time,
            'data_points': 100,
            'forecast_date': datetime.now() - timedelta(days=np.random.randint(0, 30))
        })
        
        panel_count += 1
        if panel_count % 20 == 0:
            print(f"  Generated {panel_count}/{len(products) * len(models)} panels...")
        
        plt.close(fig)  # Free memory

print(f"\n✓ Generated {panel_count} forecast visualizations")

  Generated 20/80 panels...
  Generated 40/80 panels...
  Generated 60/80 panels...
  Generated 80/80 panels...

✓ Generated 80 forecast visualizations


## 3. Create Trelliscope Display

In [4]:
# Create DataFrame
df = pd.DataFrame(data_rows)

print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df[['product', 'category', 'model', 'rmse', 'mae']].head(10))


DataFrame shape: (80, 10)

Columns: ['panel', 'product', 'category', 'model', 'rmse', 'mae', 'mape', 'training_time', 'data_points', 'forecast_date']

First few rows:
     product     category    model       rmse       mae
0  Product_A  Electronics    ARIMA   6.835162  5.869346
1  Product_A  Electronics  Prophet   3.331693  2.847957
2  Product_A  Electronics      ETS  10.045832  8.782582
3  Product_A  Electronics  XGBoost   5.980581  5.092127
4  Product_B      Apparel    ARIMA   6.885577  5.764944
5  Product_B      Apparel  Prophet   3.223626  2.723935
6  Product_B      Apparel      ETS   9.485843  8.344069
7  Product_B      Apparel  XGBoost   5.619643  4.724362
8  Product_C         Food    ARIMA   6.343052  5.483523
9  Product_C         Food  Prophet   3.241291  2.775687


In [5]:
# Create Trelliscope display
display = (
    Display(df, name="multi_model_forecast_comparison", description="Compare forecasting performance across ARIMA, Prophet, ETS, and XGBoost models for 20 products")
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=3, nrow=2)  # Start with 3×2 grid
    .set_default_labels(["product", "model", "rmse", "mae"])  # Show key info
    # Sort by best performance first
)


## 4. Launch Interactive Viewer

In [ ]:
# Launch viewer
from trelliscope.dash_viewer import create_dash_app

# app = create_dash_app(display)

# In your notebook cell:
app = create_dash_app(display, debug=True, force_write=True)
app.run(debug=True, port=8055)

print("\n" + "="*70)
print("🚀 LAUNCHING INTERACTIVE VIEWER")
print("="*70)
print(f"\n📊 Display: {display.name}")
print(f"📈 Total Panels: {len(df)}")
print(f"🎯 Products: {df['product'].nunique()}")
print(f"🤖 Models: {', '.join(df['model'].unique())}")
print("\n🌐 Opening browser on http://localhost:8053...\n")

app.run()

Rendering 80 panels...
  Rendered panel 0: 0.png
  Rendered panel 1: 1.png
  Rendered panel 2: 2.png
  Rendered panel 3: 3.png
  Rendered panel 4: 4.png
  Rendered panel 5: 5.png
  Rendered panel 6: 6.png
  Rendered panel 7: 7.png
  Rendered panel 8: 8.png
  Rendered panel 9: 9.png
  Rendered panel 10: 10.png
  Rendered panel 11: 11.png
  Rendered panel 12: 12.png
  Rendered panel 13: 13.png
  Rendered panel 14: 14.png
  Rendered panel 15: 15.png
  Rendered panel 16: 16.png
  Rendered panel 17: 17.png
  Rendered panel 18: 18.png
  Rendered panel 19: 19.png
  Rendered panel 20: 20.png
  Rendered panel 21: 21.png
  Rendered panel 22: 22.png
  Rendered panel 23: 23.png
  Rendered panel 24: 24.png
  Rendered panel 25: 25.png
  Rendered panel 26: 26.png
  Rendered panel 27: 27.png
  Rendered panel 28: 28.png
  Rendered panel 29: 29.png
  Rendered panel 30: 30.png
  Rendered panel 31: 31.png
  Rendered panel 32: 32.png
  Rendered panel 33: 33.png
  Rendered panel 34: 34.png
  Rendered panel 

[DEBUG FILTERS] Total filterable_metas: 9
[DEBUG FILTERS] Meta varnames: ['category', 'data_points', 'forecast_date', 'mae', 'mape', 'model', 'product', 'rmse', 'training_time']
[DEBUG FILTERS] Meta types: ['factor', 'number', 'time', 'number', 'number', 'factor', 'factor', 'number', 'number']
[DEBUG FILTERS] cog_data columns: ['product', 'category', 'model', 'rmse', 'mae', 'mape', 'training_time', 'data_points', 'forecast_date', 'panelKey', 'panel', 'category_label', 'model_label', 'product_label', '_panel_full_path', '_panel_type']
[DEBUG FILTERS] Factor category: using label column category_label
[DEBUG FILTERS] >>> About to call create_filter_component for category (type: factor)
[DEBUG FILTERS] >>> create_filter_component returned for category: <class 'dash.dcc.Dropdown.Dropdown'>
[DEBUG FILTERS] Created filter component for category (type: factor)
[DEBUG FILTERS] category dropdown has 4 options
[DEBUG FILTERS] category component id: {'type': 'filter', 'varname': 'category'}
[DEBU

[DEBUG] ===== Callback triggered =====
[DEBUG] Triggered ID: None
[DEBUG] Triggered prop: None
[DEBUG] Current page from state: 1, from store: 1
[DEBUG] Input values - prev_clicks: None, next_clicks: None
[DEBUG] Pagination state:
  Current page: 1
  Total pages: 14
  Total panels: 80
  Panels per page: 6
  Page data length: 6
  Panel range: 1-6


127.0.0.1 - - [15/Nov/2025 09:46:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2025 09:46:38] "POST /_dash-update-component HTTP/1.1" 204 -


[DEBUG] ===== Callback triggered =====
[DEBUG] Triggered ID: next-page-btn
[DEBUG] Triggered prop: next-page-btn.n_clicks
[DEBUG] Current page from state: 1, from store: 1
[DEBUG] Input values - prev_clicks: None, next_clicks: 1
[DEBUG] Next page clicked. New page: 2, Total pages: 14
[DEBUG] Pagination state:
  Current page: 2
  Total pages: 14
  Total panels: 80
  Panels per page: 6
  Page data length: 6
  Panel range: 7-12


...

[DEBUG] ===== Callback triggered =====
[DEBUG] Triggered ID: {'type': 'filter', 'varname': 'category'}
[DEBUG] Triggered prop: {"type":"filter","varname":"category"}.value
[DEBUG] Current page from state: 1, from store: 1
[DEBUG] Input values - prev_clicks: None, next_clicks: None
[DEBUG] Pagination state:
  Current page: 1
  Total pages: 4
  Total panels: 20
  Panels per page: 6
  Page data length: 6
  Panel range: 1-6


127.0.0.1 - - [15/Nov/2025 09:48:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2025 09:48:39] "POST /_dash-update-component HTTP/1.1" 204 -


[DEBUG] ===== Callback triggered =====
[DEBUG] Triggered ID: next-page-btn
[DEBUG] Triggered prop: next-page-btn.n_clicks
[DEBUG] Current page from state: 1, from store: 1
[DEBUG] Input values - prev_clicks: None, next_clicks: 1
[DEBUG] Next page clicked. New page: 2, Total pages: 4
[DEBUG] Pagination state:
  Current page: 2
  Total pages: 4
  Total panels: 20
  Panels per page: 6
  Page data length: 6
  Panel range: 7-12


127.0.0.1 - - [15/Nov/2025 09:48:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2025 09:48:40] "POST /_dash-update-component HTTP/1.1" 204 -


[DEBUG] ===== Callback triggered =====
[DEBUG] Triggered ID: next-page-btn
[DEBUG] Triggered prop: next-page-btn.n_clicks
[DEBUG] Current page from state: 2, from store: 2
[DEBUG] Input values - prev_clicks: None, next_clicks: 2
[DEBUG] Next page clicked. New page: 3, Total pages: 4
[DEBUG] Pagination state:
  Current page: 3
  Total pages: 4
  Total panels: 20
  Panels per page: 6
  Page data length: 6
  Panel range: 13-18


## 5. Feature Testing Guide

### ✅ Dynamic Layout Controls (Feature 1)

**Location**: Sidebar → Layout section

**Try This**:
1. Adjust **ncol slider** from 3 to 5
2. Click **"Apply Layout"**
3. Observe grid now shows 5 columns per row
4. Adjust **nrow slider** from 2 to 3 (15 panels per page)
5. Click **"Apply Layout"** again
6. Try **"Reset to Default"** to restore 3×2 layout

**Expected**: Grid re-renders with new dimensions, panels-per-page counter updates

---

### ✅ Label Configuration (Feature 2)

**Location**: Sidebar → Labels section

**Try This**:
1. **Uncheck** "training_time" and "data_points" from label list
2. Observe these labels disappear from under panels
3. Click **"Clear All"** button
4. Observe all labels disappear
5. Click **"Select All"** button
6. Observe all labels reappear
7. Keep only: product, model, rmse, mae

**Expected**: Label changes apply immediately without needing "Apply" button

---

### ✅ Filters (Built-in + Phase 4)

**Location**: Sidebar → Filters section

**Try This**:
1. **Category filter**: Select "Electronics" only
   - Observe only Electronics products shown
2. **Model filter**: Select "XGBoost" and "Prophet"
   - Observe only these 2 models shown
3. **RMSE range**: Set max to 3.0
   - Observe only low-error forecasts shown
4. **MAE range**: Set max to 2.5
   - Observe further filtering
5. **Clear filters** using individual X buttons or "Clear All"

**Expected**: Filters apply immediately, panel count updates, empty state if no matches

---

### ✅ Sorts (Built-in + Phase 4)

**Location**: Sidebar → Sorts section

**Try This**:
1. Default sort is by **RMSE (ascending)** - best models first
2. Click **"Clear All Sorts"**
3. Add sort: **training_time (ascending)** - fastest models first
4. Observe panels re-order
5. Add second sort: **RMSE (ascending)** 
6. Observe multi-column sorting (fast models, then best RMSE)

**Expected**: Panels re-order immediately, primary sort takes precedence

---

### ✅ Views (Feature from Phase 3)

**Location**: Sidebar → Views section

**Try This**:
1. **Create "Best Models" view**:
   - Filter: RMSE < 3.0
   - Sort: RMSE (ascending)
   - Labels: product, model, rmse, mae
   - Enter view name: "Best Models"
   - Click **"Save View"**
   - Observe success toast notification

2. **Clear all filters** and change layout

3. **Load saved view**:
   - Select "Best Models" from dropdown
   - Click **"Load View"**
   - Observe all filters, sorts, labels restored

4. **Create "Fast Models" view**:
   - Filter: training_time < 1.0
   - Sort: training_time (ascending)
   - Save as "Fast Models"

**Expected**: Views persist state, can switch between views, delete works

---

### ✅ Global Search (Feature from Phase 3)

**Location**: Sidebar → Search section (top)

**Try This**:
1. Search for **"Product_A"**
   - Observe only Product_A across all models (4 panels)
2. Clear search, try **"XGBoost"**
   - Observe only XGBoost model results (20 panels)
3. Try **"Electronics"**
   - Observe only Electronics category products
4. Press **"/"** key (keyboard shortcut)
   - Observe search input gains focus
5. Press **"Esc"** key
   - Observe search clears

**Expected**: Search works across all text fields, keyboard shortcuts work

---

### ✅ Export & Share (Feature 5)

**Location**: Sidebar → Export section (bottom)

**Try This**:
1. **Export filtered data**:
   - Apply filter: category = "Electronics", RMSE < 3.0
   - Click **"Export Data (CSV)"**
   - Observe CSV file downloads
   - Open CSV: should have only filtered rows, no internal columns

2. **Export view configuration**:
   - Set up complex state (filters + sorts + labels)
   - Click **"Export View (JSON)"**
   - Observe JSON file downloads
   - Open JSON: should contain full state specification

3. **Export display config**:
   - Click **"Export Config"**
   - Observe JSON file with display metadata

**Expected**: All three export types work, filenames are timestamped

---

### ✅ Keyboard Navigation (Feature 4)

**Try These Shortcuts**:
- **→** (right arrow): Next page
- **←** (left arrow): Previous page
- **/**: Focus search input
- **Esc**: Clear search or close modal
- **Click ⌨️ button** (top-right): See all keyboard shortcuts

**Expected**: All shortcuts work, keyboard help modal lists all shortcuts

---

### ✅ Help & Documentation (Feature 8)

**Try This**:
1. Click **"?" (Help) button** in top-right corner
2. Observe comprehensive help modal opens
3. Review 9 feature sections:
   - Welcome
   - Search
   - Layout Controls
   - Labels
   - Filters
   - Sorting
   - Views
   - Panel Details
   - Keyboard Shortcuts
   - Export
4. Close modal

**Expected**: Help modal is comprehensive, scrollable, closes properly

---

## 6. Performance Notes

**Dataset Size**: 80 panels (20 products × 4 models)

**Expected Performance**:
- Initial load: < 2 seconds
- Filter operations: < 300ms
- Sort operations: < 300ms
- Search: < 200ms
- Layout changes: < 400ms

**Loading States**: Should see spinners during operations

---

## 7. Key Insights to Discover

Using the interactive viewer, try to answer:

1. **Which model performs best overall?**
   - Sort by RMSE ascending
   - Look at top results

2. **Which model is fastest?**
   - Sort by training_time ascending
   - Compare training times

3. **Best model for Electronics category?**
   - Filter: category = "Electronics"
   - Sort: RMSE ascending
   - Check top model

4. **Which products are hardest to forecast?**
   - Sort by RMSE descending
   - Look at products appearing at top across models

5. **Best speed/accuracy trade-off?**
   - Filter: RMSE < 3.0, training_time < 1.0
   - See which models meet both criteria

---

## Summary

This example demonstrates:
- ✅ Layout controls for flexible viewing
- ✅ Label configuration for focused analysis
- ✅ Multi-criteria filtering for subsetting
- ✅ Multi-column sorting for ranking
- ✅ Views for saving analysis states
- ✅ Search for quick finding
- ✅ Export for sharing results

**Next**: Try Example 2 (Hyperparameter Tuning) for more Phase 4 features!